In [448]:
import os
workDir = os.getcwd()
print(workDir)
user = 'mkilpatr'

/Users/Matthew/Documents/SchoolWork/physics/GradNotes/ParticlePhysics/PostdocResearch/HGC/EstToolsSUSY/HGCStudies


In [449]:
import io
import pandas as pd
import numpy as np

sensorfile = workDir + '/Fiducials/sensorfiducialtrialrun.xls'
pcbfile = workDir + '/Fiducials/pcbfiducialtrialrun.xls'
fiducialfile = workDir + '/Fiducials/GantryTrayFiducialData.xls'
readPCB = False

if os.path.exists(sensorfile):
    dfs = pd.read_excel(sensorfile)
    dfs = dfs.dropna(how='all')
else:
    print('FILEPATHS MAY BE WRONG!! CANNOT FIND SENSOR EXCEL FILE')
    
if os.path.exists(fiducialfile):
    dff = pd.read_excel(fiducialfile)
    dff = dff.dropna(how='all')
else:
    print('FILEPATHS MAY BE WRONG!! CANNOT FIND FIDUCIAL EXCEL FILE')

if os.path.exists(pcbfile):
    dfp = pd.read_excel(pcbfile)
    dfp = dfp.dropna(how='all')
    readPCB = True
else:
    print('FILEPATHS MAY BE WRONG!! CANNOT FIND PCB EXCEL FILE')

FILEPATHS MAY BE WRONG!! CANNOT FIND PCB EXCEL FILE


In [450]:
def readComponentFiducials(df):
    X = df.index[dfs['Unnamed: 3'] == 'X'].tolist()
    Points = []
    for x in X:
        if 'Fiducial' in df['Unnamed: 2'][x]:
            Points.append((float(df['Unnamed: 4'][x]), float(df['Unnamed: 4'][x + 1])))       

    # Function to sort hte list by second item of tuple
    def Sort_Tuple(tup): 
  
        # reverse = True (Sorts in descending order for Y coordinate)
        # reverse = None (Sorts in ascending order for X coordinate)
        # key is set to sort using first or second element of 
        # sublist lambda has been used 
        tup.sort(key = lambda x: x[1], reverse=True)
        tup1, tup2 = tup[:2], tup[2:]
        tup1.sort(key = lambda x: x[0])
        tup2.sort(key = lambda x: x[0])
    
        new = [tup1[0], tup2[0], tup1[1], tup2[1]]
    
        return new 

    Points = Sort_Tuple(Points)
    return Points

sPoints = readComponentFiducials(dfs)
if readPCB:
    pPoints = readComponentFiducials(dfp)

In [451]:
#Get index of all points that correspond to the two reference frames
trayIdx = dff.index[dff['Unnamed: 2'] == 'TrayFiducial2Fine2.X'].tolist()[0]
P1IdxCenter = dff.index[dff['Unnamed: 2'] == 'P1CenterPin.X'].tolist()[0]
P1IdxOffCenter = dff.index[dff['Unnamed: 2'] == 'P1OffcenterPin.X'].tolist()[0]
P2IdxCenter = dff.index[dff['Unnamed: 2'] == 'P2CenterPin.X'].tolist()[0]
P2IdxOffCenter = dff.index[dff['Unnamed: 2'] == 'P2OffcenterPin.X'].tolist()[0]

#Organize such that the Center is always the first element
#Off center is always the second element
trayPoints = [(0., 0.), (float(dff['Unnamed: 3'][trayIdx]), float(dff['Unnamed: 3'][trayIdx + 1]))]
P1 = [(float(dff['Unnamed: 3'][P1IdxCenter]),    float(dff['Unnamed: 3'][P1IdxCenter + 1])), 
      (float(dff['Unnamed: 3'][P1IdxOffCenter]), float(dff['Unnamed: 3'][P1IdxOffCenter + 1]))]
P2 = [(float(dff['Unnamed: 3'][P2IdxCenter]),    float(dff['Unnamed: 3'][P2IdxCenter + 1])), 
      (float(dff['Unnamed: 3'][P2IdxOffCenter]), float(dff['Unnamed: 3'][P2IdxOffCenter + 1]))]

In [452]:
#Calculate the distance between two points or the origin
def distance(tup):
    if np.array(tup).size == 2:
        return np.sqrt(((tup[0])**2) + ((tup[1])**2))
    else:
        return np.sqrt(((tup[1][0] - tup[0][0])**2) + ((tup[1][1] - tup[0][1])**2))

#Using law of cosines we calculate the angle between the fiducials and the P1/P2 frames
def angleOfFrames(Fiducial, baseplate):
    f = distance(Fiducial)
    b = distance(baseplate)
    th1 = 0.
    if (baseplate[0][0] - baseplate[1][0]) > 0.:
        b1, b2 = (baseplate[0][0] - baseplate[1][0]), (baseplate[0][1] - baseplate[1][1])
    else:
        b1, b2 = (baseplate[1][0] - baseplate[0][0]), (baseplate[1][1] - baseplate[0][1])
        
    th1 = Fiducial[1][0]*b1 + Fiducial[1][1]*b2
    theta = np.pi/2 - np.arccos((th1)/(f*b))
    return theta

#Using a rotation matrix transform the measurements of the points in the fiducial frame
def transform(points, theta, origin):
    c, s = np.cos(theta), np.sin(theta)
    rotate = np.array(((c, -s), (s, c)))
    return np.dot((points - origin), rotate.T)

#Calculate the angle offset of the placement of the components
#also get the center of the square that is made by the fiducial measurements
def angleDeviation(coord):
    rad = []
    deg = []
    for x in range(0, int(len(coord)/2)):
        r = np.arctan2((coord[x*2 + 1][1] - coord[x*2][1]), (coord[x*2 + 1][0] - coord[x*2][0]))
        rad.append(r)
        deg.append(np.degrees(r))
        
    center = np.sum(coord, axis=0)/4
    
    return rad, deg, center

In [453]:
#Calculate the angle of the two tray positions P1, P2 with respect to the fiducials on the carrying tray
angleP1 = angleOfFrames(trayPoints, P1)
angleP2 = angleOfFrames(trayPoints, P2)

#Transform the coordinates of the Sensor/PCB fiducial measurements to the tray position frame P1
transformSensorP1 = transform(np.array(sPoints), angleP1, np.array(P1[0]))
if readPCB:
    transformPCBP1 = transform(np.array(pPoints), angleP1, np.array(P1[0]))

#Transform the coordinates of the Sensor/PCB fiducial measurements to the tray position frame P2
####NOTICE currently using the measurements from the P1 tray as a placeholder
transformSensorP2 = transform(np.array(sPoints), angleP2, np.array(P2[0]))
if readPCB:
    transformPCBP2 = transform(np.array(pPoints), angleP2, np.array(P2[0]))

In [461]:
#Calculate the angle offset (radians, degrees) of the sensor/PCB with respect to the P1 Tray
#Calculate the X/Y center position of the sensor/PCB with respect to the P1 Tray
radSensorP1, degSensorP1, centerSensorP1 = angleDeviation(transformSensorP1)
if readPCB:
    radPCBP1, degPCBP1, centerPCBP1 = angleDeviation(transformPCBP1)

#Calculate the angle offset (radians, degrees) of the sensor/PCB with respect to the P2 Tray
#Calculate the X/Y center position of the sensor/PCB with respect to the P2 Tray
radSensorP2, degSensorP2, centerSensorP2 = angleDeviation(transformSensorP2)
if readPCB:
    radPCBP2, degPCBP2, centerPCBP2 = angleDeviation(transformPCBP2)
    radPCB = {'P1': radPCBP1, 'P2': radPCBP2}
    degPCB = {'P1': degPCBP1, 'P2': degPCBP2}
    centerPCB = {'P1': distance(centerPCBP1), 'P2': distance(centerPCBP2)}
    
radSensor = {'P1': radSensorP1, 'P2': radSensorP2}
degSensor = {'P1': degSensorP1, 'P2': degSensorP2}
centerSensor = {'P1': distance(centerSensorP1), 'P2': distance(centerSensorP2)}

In [463]:
#Classify the placement of each component
degrees = [89.5, 89.0, 88.0, 0.]
Classify = ['GOOD', "OK", "BAD", "VERY BAD"]
centers = [0.025, 0.050, 0.100, 5.0]
place = ['P1', 'P2']

angleSensorNotes = {'P1': '', 'P2': ''}
anglePCBNotes = {'P1': '', 'P2': ''}
centerSensorNotes = {'P1': '', 'P2': ''}
centerPCBNotes = {'P1': '', 'P2': ''}
for p in place:
    for i, d in enumerate(degrees):
        if abs(degSensor[p][0]) > degrees[i] or abs(degSensor[p][1]) > degrees[i]:
            angleSensorNotes[p] = "The angle of the sensor is " + Classify[i] + " in position " + p
        
        if centerSensor[p] < centers[i]:
            centerSensorNotes[p] = "The placement of the sensor is " + Classify[i] + " in position " + p
        if readPCB:
            if (abs(degPCB[p][0]) > degrees[i] or abs(degPCB[p][1]) > degrees[i]):
                anglePCBNotes[p] = "The angle of the PCB is " + Classify[i] + ' in position ' + p
            if centerSensor[p] < centers[i]:
                centerPCBNotes[p] = "The placement of the PCB is " + Classify[i] + " in position " + p
                
    if 'BAD' in angleSensorNotes[p]:
        print(angleSensorNotes[p])
    if 'BAD' in anglePCBNotes[p]:
        print(anglePCBNotes[p])

The angle of the sensor is VERY BAD in position P1
The angle of the sensor is VERY BAD in position P2
